Builiding a chatbot with message history

In [2]:
# so we will use goq api for that we need to laod the api key 
# form dot env file to do that we will use 

import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [ ]:
# importing llm model
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E605B85090>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E605B854E0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
# we need to define system and human message

from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Sameer and I am chief ai engineer")])
# here we can see that the output is not in 
# proper form so we need to create a parser str output parser

AIMessage(content="Hello Sameer, it's nice to meet you!\n\nThat's a fascinating title - Chief AI Engineer. What kind of projects are you working on? Are you focused on a particular industry or application of AI?\n\nI'm always eager to learn more about the work people are doing in the field of artificial intelligence.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 22, 'total_tokens': 92, 'completion_time': 0.127272727, 'prompt_time': 0.002105458, 'queue_time': 0.231732171, 'total_time': 0.129378185}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c9ea4357-e653-45d3-80d3-3da9c69c8fec-0', usage_metadata={'input_tokens': 22, 'output_tokens': 70, 'total_tokens': 92})

In [9]:
from langchain_core.messages import AIMessage 
model.invoke(
    [
        HumanMessage(content="Hi,My name is Sameer and i am cheif ai engineer"),
        AIMessage(content="Hello Sameer, its exciting to work with you are a ai enginner and i am a chatbot"),
        # here we harcoded the message which i will get form ai
        HumanMessage(content="Hey whats my name and what do i do")
        # here we are trying to see if llm remember the context
    ]
)
# as we gave our response in form of list so it rembered the messgage

AIMessage(content="You told me your name is Sameer, and you said you are a Chief AI Engineer!  \n\nIs there anything else you'd like to tell me about your work? I'm eager to learn more. 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 62, 'total_tokens': 111, 'completion_time': 0.089090909, 'prompt_time': 0.003606549, 'queue_time': 0.236766009, 'total_time': 0.092697458}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-452f6beb-4ffb-4d01-b63f-548efd3b8304-0', usage_metadata={'input_tokens': 62, 'output_tokens': 49, 'total_tokens': 111})

### Message History

We can use Message History class to wrap our model and make it stateful and theis will keep track of inputs and otputs of the model


In [15]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import  BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Here we imported all model which will me Message History
# here how we will differntiate in sessions
# making a function for that 

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(model,get_session_history)
# this has model+history

In [16]:
config={"configurable":{"session_id":"chat1"}}

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi I am sameer and i an ai engineer")],
    config=config
)
response.content

"Hi Sameer! \n\nIt's great to meet another AI enthusiast!  \n\nWhat area of AI engineering are you most passionate about?  Do you work on things like natural language processing, computer vision, or something else entirely?  😊 \n\n"

In [ ]:
response_1=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config
)
response.content
# here it rembered beacuse of same session id

"Hi Sameer! \n\nIt's great to meet another AI enthusiast!  \n\nWhat area of AI engineering are you most passionate about?  Do you work on things like natural language processing, computer vision, or something else entirely?  😊 \n\n"

In [ ]:
# now lets change config
config_1={"configurable":{"session_id":"chat2"}}
response_2=with_message_history.invoke(
    HumanMessage(content="Whats my name"),
    config=config_1
)
response_2.content
# here model is not able to remember the name

"Since I don't have access to past conversations, I don't know your name.  \n\nWould you like to tell me? 😊  \n"

In [25]:
# now lets change config
config_1={"configurable":{"session_id":"chat2"}}
response_2=with_message_history.invoke(
    HumanMessage(content="My name is sameer now you recall"),
    config=config_1
)
response_2.content
# here model is not able to remember the name

"Hello Sameer!  \n\nNice to meet you.  I'll remember that your name is Sameer. What can I help you with? 😊  \n"

### Here we understood how to use chat history now we will implement other things too

In [32]:
# prompt template

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful ai assistant answer the question to you best ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [34]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is sameer")]})

AIMessage(content="Hello Sameer, it's nice to meet you!  How can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 28, 'total_tokens': 54, 'completion_time': 0.047272727, 'prompt_time': 0.002136626, 'queue_time': 0.236435103, 'total_time': 0.049409353}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b476a711-fab4-41eb-a670-a44822644be3-0', usage_metadata={'input_tokens': 28, 'output_tokens': 26, 'total_tokens': 54})

In [36]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x000001E605BE3EB0>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [ ]:

config={"configurable":{"session_id":"chat_1"}}
response=with_message_history.invoke(
    HumanMessage(content="Hi my name is sameer"),
    config=config
)
response.content

"Hi Sameer! It's nice to meet you. 👋\n\nIs there anything I can help you with today? 🙂  \n"

In [42]:
# add more complexity

prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a helpful assistant. Answer all questions to best of your ability in {language}"
            
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [ ]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is sameer")],"language":"Hindi"})

In [62]:
response

AIMessage(content='हाँ, मैं याद रखता हूँ! आपका नाम समीर है। 😊 \n\nक्या मैं आपकी मदद करने के लिए कुछ और कर सकता हूँ?\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 154, 'total_tokens': 192, 'completion_time': 0.069090909, 'prompt_time': 0.007484004, 'queue_time': 0.238511984, 'total_time': 0.076574913}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-950d5b5f-fcff-4243-bc73-cb613f532aca-0', usage_metadata={'input_tokens': 154, 'output_tokens': 38, 'total_tokens': 192})

In [46]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [56]:
config={"configurable":{"session_id":"chat4"}}
response_2=with_message_history.invoke(
    {
        "messages":[HumanMessage(content="do you remeber my name")],"language":"Hindi"
    },config=config
)

In [61]:
response_2.content

'हाँ, मेरे लिए तो याद रखना बहुत आसान है! आपका नाम समीर है। 😊\n\nक्या आप कुछ और पूछना चाहते हैं?\n\n\n\n\n'

## Managing Conversation History

One important step to understand when building application is how to manage converstaion history If left unmanged the listof message will grwo unbounded and potentially overflow the context window of llml 

In [66]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\nande\anaconda3\envs\genai_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [76]:
# lets pass trimeer through chain

from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |trimmer
    |model 
)

chain.invoke(
    {
    "messages":messages+[HumanMessage(content="What ice cream do i like")],
    "language":"english"
    }
)
reponse.content


'नमस्ते Sameer! \n\nआपका स्वागत है! \n\nमैं आपकी मदद करने के लिए तैयार हूँ। आप मुझसे क्या पूछना चाहेंगे? 😊 \n'